In [1]:
!pip install contexto

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import json
import re

import nltk
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from contexto.correccion import Corrector, corregir_texto
from contexto.limpieza import *
from nltk.corpus import stopwords
from langdetect import detect

nltk.download('all')
stop = stopwords.words('spanish')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Pac

In [4]:
ruta_tweets = "tweets.json"

with open(ruta_tweets, "r") as fp:
    diccionario_tweets = json.load(fp)

    nombre_archivo_clasificacion = "clasificacion.txt"

    llaves = list(diccionario_tweets['data'].keys())
    tweets = []

    for i in llaves:
        tweets.append(diccionario_tweets['data'][i]['text'])

In [5]:
tweets

['quiero llorar d lo mal q m siento',
 'hyukjae dijo que no pudo dormir por estar preocupado que paso la semana pasada para que hyuk estuviera preocupado , donghae😧 les dejo el link\n\nhttps://t.co/yAqtt6u2RM',
 '@EFjvelezb @petrogustavo Ahora dilo sin llorar.',
 "@FournierChanta5 @St_Martineau @radiopirate Triste mais j'ai bien l'impression que ça va en arriver là...",
 'estoy triste.\n\n*suena party favors*\n\nautomáticamente yo: https://t.co/uennacBvZA',
 'Sobre a sexualidade de Styvenson, não me afeta em nada. Entre quatro paredes, ele pode fazer o que quiser. Acho ele um bom Senador. Defende pautas interessantes. Preocupado mesmo eu fico com os outros dois senadores esquerdistas que estão sempre fodendo com o povo Potiguar.',
 '@oliveiranalice Ele foi super vaiado nesse dia, caíram matando em cima dele, foi triste',
 '@flegaria Siiii le tocaron a él, yo medio vi pero estaba demasiado cansada y drogada 🤣',
 '@opaixaocolorada E por aí deve ter muitos …triste!',
 'Apenas cansada. É i

In [6]:
df = pd.DataFrame({"Comentarios": tweets})
df.head(10)

,Comentarios
0,quiero llorar d lo mal q m siento
1,hyukjae dijo que no pudo dormir por estar preo...
2,@EFjvelezb @petrogustavo Ahora dilo sin llorar.
3,@FournierChanta5 @St_Martineau @radiopirate Tr...
4,estoy triste.\n\n*suena party favors*\n\nautom...
5,"Sobre a sexualidade de Styvenson, não me afeta..."
6,@oliveiranalice Ele foi super vaiado nesse dia...
7,"@flegaria Siiii le tocaron a él, yo medio vi p..."
8,@opaixaocolorada E por aí deve ter muitos …tri...
9,Apenas cansada. É isso.


- Primero arrobas
- Luego links
- Luego quitar comentarios que no están en español
- Después acentos
- Luego limpieza básica

- Palabras cortas
- Corrección de palabras


### **Arrobas**

In [7]:
URL_PATTERN = r'\S*@\S+'
df.Comentarios = df.Comentarios.str.replace(URL_PATTERN, '', regex = True)
df = df[df['Comentarios'].notnull()]
df = df[df['Comentarios'] != '']
df.head()

,Comentarios
0,quiero llorar d lo mal q m siento
1,hyukjae dijo que no pudo dormir por estar preo...
2,Ahora dilo sin llorar.
3,Triste mais j'ai bien l'impression que ça v...
4,estoy triste.\n\n*suena party favors*\n\nautom...


### **Links**

In [8]:
URL_PATTERN = r'[A-Za-z0-9:]+://[A-Za-z0-9%-_]+(/[A-Za-z0-9%-_])*(#|\\?)[A-Za-z0-9%-_&=]*'
df.Comentarios = df.Comentarios.str.replace(URL_PATTERN, '', regex = True)
df = df[df['Comentarios'].notnull()]
df = df[df['Comentarios'] != '']
df.head()

,Comentarios
0,quiero llorar d lo mal q m siento
1,hyukjae dijo que no pudo dormir por estar preo...
2,Ahora dilo sin llorar.
3,Triste mais j'ai bien l'impression que ça v...
4,estoy triste.\n\n*suena party favors*\n\nautom...


### **Eliminar Tweets de otros idiomas**

In [9]:
df['Idioma'] = 0
for i in range(0,len(df)):
    temp = df.Comentarios.iloc[i]
    try:
        df['Idioma'].iloc[i] = detect(temp)
    except:
        df['Idioma'].iloc[i] = "error"

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [10]:
df.head()

,Comentarios,Idioma
0,quiero llorar d lo mal q m siento,es
1,hyukjae dijo que no pudo dormir por estar preo...,es
2,Ahora dilo sin llorar.,es
3,Triste mais j'ai bien l'impression que ça v...,fr
4,estoy triste.\n\n*suena party favors*\n\nautom...,es


In [11]:
df = df[df['Idioma'] == 'es']
df.head()

,Comentarios,Idioma
0,quiero llorar d lo mal q m siento,es
1,hyukjae dijo que no pudo dormir por estar preo...,es
2,Ahora dilo sin llorar.,es
4,estoy triste.\n\n*suena party favors*\n\nautom...,es
7,"Siiii le tocaron a él, yo medio vi pero estab...",es


### **Acentos y mayusculas**

In [12]:
trans = str.maketrans('áéíóúü', 'aeiouu')
df.Comentarios = df.Comentarios.apply(lambda x: x.translate(trans))
df.Comentarios = df.Comentarios.apply(lambda x: x.lower())
df.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,Comentarios,Idioma
0,quiero llorar d lo mal q m siento,es
1,hyukjae dijo que no pudo dormir por estar preo...,es
2,ahora dilo sin llorar.,es
4,estoy triste.\n\n*suena party favors*\n\nautom...,es
7,"siiii le tocaron a el, yo medio vi pero estab...",es


Voy a poner todas las "risas" **Jajajajj** o **Jajajjjaaaj** en el mismo formato

In [13]:
regex = r'\b[JajJA]+\b'
df.Comentarios = df.Comentarios.str.replace(regex, 'jajaja', regex = True)
df.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,Comentarios,Idioma
0,quiero llorar d lo mal q m siento,es
1,hyukjae dijo que no pudo dormir por estar preo...,es
2,ahora dilo sin llorar.,es
4,estoy triste.\n\n*suena party favors*\n\nautom...,es
7,"siiii le tocaron jajaja el, yo medio vi pero ...",es


### **Limpieza basica**

In [14]:
# Limpieza básica, se pasa todo a minúsculas, se eliminan signos de puntuación y caracteres numéricos
df.Comentarios = df.Comentarios.apply(lambda x: limpieza_basica(x))
df.head()

,Comentarios,Idioma
0,quiero llorar d lo mal q m siento,es
1,hyukjae dijo que no pudo dormir por estar preo...,es
2,ahora dilo sin llorar,es
4,estoy triste suena party favors automaticament...,es
7,siiii le tocaron jajaja el yo medio vi pero es...,es


In [15]:
# Para quitar acentos (diéresis, tildes y virgulillas)
df.Comentarios = df.Comentarios.apply(lambda x: remover_acentos(x))
df.head()

,Comentarios,Idioma
0,quiero llorar d lo mal q m siento,es
1,hyukjae dijo que no pudo dormir por estar preo...,es
2,ahora dilo sin llorar,es
4,estoy triste suena party favors automaticament...,es
7,siiii le tocaron jajaja el yo medio vi pero es...,es


In [16]:
# Quitar palabras con menos de 4 caracteres
df.Comentarios = df.Comentarios.apply(lambda x: remover_palabras_cortas(x, n_min=3))
df.head()

,Comentarios,Idioma
0,quiero llorar mal siento,es
1,hyukjae dijo que pudo dormir por estar preocup...,es
2,ahora dilo sin llorar,es
4,estoy triste suena party favors automaticamente,es
7,siiii tocaron jajaja medio pero estaba demasia...,es


In [17]:
df = df[df['Comentarios'].notnull()]
df = df[df['Comentarios'] != '']

In [18]:
import string

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    #print(tag)
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def process_tweet(tweet, stopwords_new, stem = False, lemma = True):
    """Process sentences function.
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    """
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
    tweets_clean = []
    lemmatizer = WordNetLemmatizer()

    for word in tweet_tokens:
        if (word not in stopwords_new and word not in string.punctuation and word != ""):
          lemma_word = lemmatizer.lemmatize(word, get_wordnet_pos(word))
          tweets_clean.append(lemma_word)
    return ' '.join(tweets_clean)

In [19]:
df["TweetPre"]= df.Comentarios.apply(limpieza_basica)
df["TweetPre"] = df["TweetPre"].apply(remover_acentos)
df["TweetPre"] = df["TweetPre"].apply(remover_palabras_cortas, n_min = 4)
df["TweetPre"] = df["TweetPre"].apply(process_tweet, stopwords_new = stop)
df = df[df['Comentarios'].notnull()]
df = df[df['Comentarios'] != '']
df.head()

,Comentarios,Idioma,TweetPre
0,quiero llorar mal siento,es,quiero llorar siento
1,hyukjae dijo que pudo dormir por estar preocup...,es,hyukjae dijo pudo dormir preocupado paso seman...
2,ahora dilo sin llorar,es,ahora dilo llorar
4,estoy triste suena party favors automaticamente,es,triste suena party favor automaticamente
7,siiii tocaron jajaja medio pero estaba demasia...,es,siii tocaron jajaja medio demasiado cansada dr...


In [21]:
df.to_csv("totally_processed_tweets_well.csv")

### **Palabras mal escritas**

In [22]:
corrector = Corrector('spanish')
df.Comentarios = df.Comentarios.apply(lambda x: corregir_texto(x, corrector=corrector))
df.head()

,Comentarios,Idioma,TweetPre
0,quiero llorar mal siento,es,quiero llorar siento
1,hyukjae dijo que pudo dormir por estar preocup...,es,hyukjae dijo pudo dormir preocupado paso seman...
2,ahora dilo sin llorar,es,ahora dilo llorar
4,estoy triste suena party favors automaticamente,es,triste suena party favor automaticamente
7,siiii tocaron jajaja medio pero estaba demasia...,es,siii tocaron jajaja medio demasiado cansada dr...


In [23]:
t = df.Comentarios[0]
corregir_texto(t, corrector=corrector)

'quiero llorar mal siento'

In [24]:
corrector.palabras_conocidas(t)

{'llorar', 'mal', 'quiero', 'siento'}

In [25]:
df.to_csv("totally_processed_tweets_well_complete.csv")